# Demonstrate Allocated Score / STAR-PR voting

This [Jupyter notebook](https://jupyter.org) demonstrates [Allocated Score](https://electowiki.org/wiki/Allocated_Score) voting, aka STAR-PR. It is a multi-winner [Proportional Representation](https://en.wikipedia.org/wiki/Proportional_representation)
voting method for score ballots ala
[STAR-voting](https://en.wikipedia.org/wiki/STAR_voting).


See the Python code in the GitHub repository at https://github.com/nealmcb/allocated_score_voting
It borrows the Python implementation from the [Allocated Score](https://electowiki.org/wiki/Allocated_Score) page at [electowiki](https://electowiki.org).

First, import the necessary libraries

In [1]:
import pandas as pd
import allocated_score

The default maximum score in STAR-PR is 5.

Note that 0 is also a valid score, so there are 6 possible ratings for each candidate.

In [2]:
max_score = 5

Our first example uses the data in `src/Tests/sample.test.js` from https://github.com/Equal-Vote/star-core, but looks for three winners rather than one.

Specify the number of winners to select

In [3]:
num_winners = 3

We name the candidates, separated by commas, and automatically split into a list.

In [4]:
candidates = 'Adam,Becky,Cindy,Dylan,Eliza'.split(',')

For each ballot, we include a row with the score for each candidate in turn.

Here are 7 ballots. Note e.g. that the score in the first ballot for Cindy is a 5.

In [5]:
scores = [
    [0,0,5,3,2],
    [4,0,3,3,2],
    [0,0,0,3,1],
    [2,0,0,3,4],
    [0,0,0,0,0],
    [1,5,0,3,5],
    [0,0,0,0,0],
]

We read the data into a [Pandas](https://pandas.pydata.org/) dataframe, so we can use its powerful capabilities in the implementation.

In [6]:
S = pd.DataFrame.from_records(scores, columns=candidates)

In [7]:
S

,Adam,Becky,Cindy,Dylan,Eliza
0,0,0,5,3,2
1,4,0,3,3,2
2,0,0,0,3,1
3,2,0,0,3,4
4,0,0,0,0,0
5,1,5,0,3,5
6,0,0,0,0,0


Finally, we determine the winners using the provided library function.

In [8]:
allocated_score.Allocated_Score(max_score, num_winners, S)

['Dylan', 'Eliza', 'Adam']

## Simplified setup of factions and trying various winner counts
Here we see how to set up an election with *factions*, each with as many candidates as there are seats, where voters are very simplistically modeled into `profiles` showing how they score the factions, and in which everyone ranks every candidate in the faction the same way.

In [9]:
num_winners = 5

In [10]:
factions = ['A', 'B', 'C']
candidates = allocated_score.dup_factions(factions, num_winners)

In [11]:
red   = [5, 0, 0]
green = [0, 4, 5]
blue  = [0, 3, 0]

In [12]:
profiles = [(red, 21), (green, 41), (blue, 38)]

With the blue profile of voters giving a score of 3 to the `B` faction, they get 4 winners:

In [13]:
allocated_score.tabulate_factions(max_score, num_winners, factions, candidates, profiles)

Example election: num_winners=5, max_score=5, factions=['A', 'B', 'C']
Profiles:
  21: [5, 0, 0]
  41: [0, 4, 5]
  38: [0, 3, 0]
Winners: ['B1', 'B2', 'B3', 'A1', 'B4']


Note here that if blue gives a higher score of 5 to `B`, they only get 3 winners:

In [14]:
blue5  = [0, 5, 0]
profiles5 = [(red, 21), (green, 41), (blue5, 38)]
allocated_score.tabulate_factions(max_score, num_winners, factions, candidates, profiles5)

Example election: num_winners=5, max_score=5, factions=['A', 'B', 'C']
Profiles:
  21: [5, 0, 0]
  41: [0, 4, 5]
  38: [0, 5, 0]
Winners: ['B1', 'B2', 'B3', 'A1', 'C1']


This is certainly counter-intuitive, and is related to issues of "non-monotonicity" in single-winner methods.

But with multiple winners, that sort of issue seems trickier to define and even harder to avoid.